In [1]:
import os
import math
from scipy import integrate
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# read of the geometry from a data file
naca_filepath = os.path.join('resources', 'naca0012.dat')
with open(naca_filepath,'r') as file_name:
    x, y = numpy.loadtext(file_name, dtype=float, delimiter='\t', unpack=True)
    
# plot the geometry
width = 10
plt.figure(figsize=(width, width))
plt.xlabel('x', fontsize=16)
plt.ylabel('y', fontsize=16)
plt.grid()
plt.plot(x, y, color='k', linestyle='-', linewidth=2)
plt.axis('scaled', adjustable='box')

FileNotFoundError: [Errno 2] No such file or directory: 'resources/naca0012.dat'

In [3]:
class Panel:
    """
    Contains information related to a panel.
    """
    def __init__(self, xa, ya, xb, yb):
        """
        Initializes the panel.
        
        Sets the end-points and calculates the center, length,
        and angle (with the x-axis) of the panel.
        Defines if the panel is on the lower or upper surface of the geometry.
        Initializes the source-sheet strength, tangential velocity,
        and pressure coefficient to zero.
        
        Parameters
        ----------
        xa: float
            x-coordinate of the first end-point
        ya: float
            y-coordinate of the first end-point
        xb: float
            x-coordinate of the second end-point
        yb: float
            y-coordinate of the second end-point
        """
        self.xa, self.ya = xa, ya
        self.xb, self.yb = xb, yb
        
        self.xc, self.yc = (xa+xb)/2, (ya+yb)/2       # control(center) point of the panel
        self.length = ((xb-xa)**2+(yb-ya)**2)         # length of the panel
        
        # orientation of the panel (angle with x-axis)
        if xb-xa <=0.:
            self.beta = math.acos((yb-ya)/self.length)
        elif xb-xa > 0.:
            self.beta = math.pi + math.acos(-(yb-ya)/self.length)
            
        # location of the panel
        if self.beta <= math.pi:
            self.loc = 'upper'
        else:
            self.loc = 'lower'
            
        self.sigma = 0.                           # source strength
        self.vt = 0.                              # tangential velocity
        self.cp = 0.                              # coefficient of pressure

## Define the panels by creating a circle around the airfoil and marking the x-coordinates. The y-coordinates are found by projecting them from the circle onto the airfoil through interpolation. Panels are smaller near leading and trailing edges due to more curvature.

In [4]:
def define_panels(x, y, N=40):
    """
    Descretizes the geometry into panels using the 'cosine' method.
    
    Parameters
    ----------
    x: 1D array of floats
        x-coordinate of the points defining the geometry
    y: 1D array of floats
        y-coordinate of the points defining the geometry
    N: integer, optional
        Number of panels;
        default: 40.
        
    Returns
    -------
    panels: 1D numpy array of Panel objects
        the discretization of the geometry into panels
    """
    R = (x.max()-x.min())/2                                       # Radius of the circle
    x_center = (x.max()+x.min())/2                                # x-coord of the center
    x_circle = x_center + R*np.cos(np.linspace(0, 2*math.pi, N+1))  # x-coord of the circle points
    
    x_ends = np.copy(x_circle)                     # projection of the x-coord on the surface
    y_ends = np.empty_like(x_ends)                 # intiialization of the y-coord numpy array
    
    x, y = np.append(x, x[0]), np.append(y, y[0])  # extend arrays using numpy.append
    
    # computes the y-coordinate of the end-points
    I = 0
    for i in range(N):
        while I < len(x)-1:
            if (x(I) <= x_ends[i] <= x[I+1]) or (x[I+1] <= x_ends[i] <= x[I]):
                break
            else:
                I += 1
        a = (y[I+1] - y[I])/(x[I+1] - x[I])
        b = y[I+1] - a*x[I+1]
    y_ends[i] = y_ends[0]
    
    panels = np.empty(N, dtype = object)
    for i in range(N):
        panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    
    return panels